In [ ]:
import numpy as np
import h5py
import setGPU

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
import math

from datetime import datetime
from tensorboard import program
import os
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline

from functions import preprocess_anomaly_data,\
roc_objective,load_model, save_model, make_mse_loss
import pickle
from autoencoder_classes import AE

## Load data

In [ ]:
# Data = (N,19,3,1).flatten()
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler = pickle.load(f)

## Define autoencoder model

In [ ]:
latent_dim = 3
input_shape = 57

In [ ]:
#encoder
inputArray = Input(shape=(input_shape))
x = BatchNormalization()(inputArray)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dense(16, kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
encoder = Dense(latent_dim, kernel_initializer=tf.keras.initializers.HeUniform())(x)
# x = BatchNormalization()(x)
# encoder = LeakyReLU(alpha=0.3)(x)
#decoder
x = Dense(16, kernel_initializer=tf.keras.initializers.HeUniform())(encoder)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
decoder = Dense(input_shape, kernel_initializer=tf.keras.initializers.HeUniform())(x)

#create autoencoder
autoencoder = Model(inputs = inputArray, outputs=decoder)
autoencoder.summary()

In [ ]:
ae = AE(autoencoder)
ae.compile(optimizer=keras.optimizers.Adam(lr=0.00001))

## Train model

In [ ]:
EPOCHS = 150
BATCH_SIZE = 1024

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from neptunecontrib.monitoring.keras import NeptuneMonitor
callbacks=[]
callbacks.append(ReduceLROnPlateau(monitor='val_loss',  factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
callbacks.append(TerminateOnNaN())
callbacks.append(NeptuneMonitor())
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=10, restore_best_weights=True))

In [ ]:
print("TRAINING")
history = ae.fit(X_train_flatten, X_train_scaled, epochs = EPOCHS, batch_size = BATCH_SIZE,
                  validation_split=0.2,
                  callbacks=callbacks)

In [ ]:
save_model('AE_models/final_models/withCorrectPrefiltering/AE_notpruned', ae.autoencoder)

In [ ]:
ae.save('AE_Delphes')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
plt.plot(history.history['loss'][:], label='Training loss')
plt.plot(history.history['val_loss'][:], label='Validation loss')
plt.title('Training and validation loss - MSE')
#plt.yscale('log', nonposy='clip')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

## Prediction - background

In [ ]:
qcd_prediction = ae.autoencoder(X_test_flatten)

## Prediction - Beyond Standard Model samples

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']

In [ ]:
bsm_results = []

for i, label in enumerate(bsm_labels):
    bsm_prediction = ae.autoencoder(bsm_data[i])
    
    bsm_results.append([label, bsm_target[i], bsm_prediction])

In [ ]:
output_result = '/eos/user/e/epuljak/forDelphes/CorrectDataResults/AE_result_notpruned.h5'

In [ ]:
h5f = h5py.File(output_result, 'w')
h5f.create_dataset('QCD_input', data=X_test_flatten)
h5f.create_dataset('QCD_target', data=X_test_scaled)
h5f.create_dataset('predicted_QCD', data = qcd_prediction)
for i, bsm in enumerate(bsm_results):
    h5f.create_dataset('%s_scaled' %bsm[0], data=bsm[1])
    h5f.create_dataset('%s_input' %bsm[0], data=bsm_data[i])
    h5f.create_dataset('predicted_%s' %bsm[0], data=bsm[2])

h5f.close()